In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import queue
%matplotlib inline

# import data
# import config
# import process

In [ ]:
# log_path=os.getcwd()+'/data/driving_log.csv'
# # img_path=os.getcwd()+'/data/IMG/*.jpg'

# ####read data#########
# ######################

# from numpy import genfromtxt
# def decode_filename(filename):
# 	return os.getcwd()+'/data/'+filename.decode("utf-8").strip()

# files = genfromtxt(log_path,delimiter=',',dtype="|S50, |S50, |S50, float, float, float, float")
# files=files[1:]
# files = np.array(files)
# src_img_names=[]
# src_steering_angles=[]


# for line in files:
# 	steering_angle = line[3]
# 	center_name= decode_filename(line[0])
# 	left= decode_filename(line[1])
# 	right= decode_filename(line[2])
# 	src_img_names.append(center_name)
# 	src_steering_angles.append(steering_angle)
# 	src_img_names.append(left)
# 	src_steering_angles.append(steering_angle+CAMERA_OFFSET)
# 	src_img_names.append(right)
# 	src_steering_angles.append(steering_angle-CAMERA_OFFSET)

# #shuffle data to avoid overfitting
# from sklearn.utils import shuffle
# src_img_names,src_steering_angles= shuffle(src_img_names,src_steering_angles)
# #reformat list as np array
# src_img_names=np.array(src_img_names)
# src_steering_angles=np.array(src_steering_angles)

# print("data size:",src_img_names.shape[0])
dataset = data.DataSet([("../data/driving_log.csv", "data/IMG/")])
print(dataset.size())

In [ ]:
log = pd.read_csv(config.train_data[0][0])
log.columns = ['center', 'left', 'right', 'steer', 'throttle', 'brake', 'speed']

plt.figure();
log.hist(column='steer',bins=101,alpha=0.5)
plt.show()

In [ ]:
len_plt = 3500

steers_input= np.array(log.steer,dtype=np.float32)

t_s = np.arange(len(steers_input))
x_s = np.array(log.steer)
y_s = steers_input

plt.plot(t_s[0:len_plt],x_s[0:len_plt],'--',t_s[0:len_plt],y_s[0:len_plt]);
plt.xlabel('time')
plt.ylabel('steer')
plt.ylim(-1,1);

log['steer'] = pd.Series(steers_input, index=log.steer.index)

In [ ]:
i = np.random.randint(dataset.size())
print(i)

row = dataset.log.iloc[[i]].reset_index()
queen_command = queue.Queue()

path_pre = '../'
imagefile_path_c = path_pre + row.center[0]
imagefile_path_l = path_pre + row.left[0]
imagefile_path_r = path_pre + row.right[0]
steer = float(row.steer[0])

commands = {'shift': 'shift', 'resize': 'resize', 'flip': 'flip','none': 'none','brightness' : 'brightness'}

In [ ]:
def _shift_image(image,steer,lane_id):    
    # Translation
	steer = float(steer)
	rows,cols = image.shape[:2]
	trans_range = config.shift_range
	tr_x = trans_range*np.random.uniform()-trans_range/2
	steer_ang = steer + tr_x/trans_range*2*.2
	# steer_ang = steer + tr_x/trans_range*2*.3	
	tr_y = 10*np.random.uniform()-10/2

	Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
	image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))

	return image_tr,steer_ang,tr_x

def _brightness_image(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def _crop_and_resize_image(image):
    # Preprocessing image files
    col,row,ch = config.image_size
    cropped_image = image[55:135, :, :]    
    shape = cropped_image.shape
    # note: numpy arrays are (row, col)!
    image = cropped_image[math.floor(shape[0]/4):shape[0]-25, 0:shape[1]]
    image = cv2.resize(image,(col,row), interpolation=cv2.INTER_AREA)    
    return image     

def _flip_image(image,steer):
	steer = float(steer)
	prob_f = np.random.random()
	if prob_f > 0.5:
		# # flip the image and reverse the steering angle
		steer_out = -1*steer
		image_out =  cv2.flip(image, 1)
	else:
		image_out, steer_out = image, steer
	return image_out, steer_out

def plot_images(images,steers,titles,deg = 0):
    plt.subplot(1,3,1)
    plt.imshow(images[0]);
    plt.axis('on')    
    if deg==0:
        plt.title(titles[0] + ' ' +'steer:'+ str(np.round(steers[0],2) ) );        
    else:
        plt.title('Steer:'+ str((np.round((steer+.1)*180/np.pi,2) )))
    plt.subplot(1,3,2)
    plt.imshow(images[1]);
    plt.axis('on')        
    if deg==0:
        plt.title('Steer:'+ str(np.round(steers[1],2) ));
        plt.title(titles[1] + ' ' +'steer:'+ str(np.round(steers[1],2) ) );                
    else:
        plt.title('Steer:'+ str(np.round(steer*180/np.pi,2) ));
    plt.subplot(1,3,3)
    plt.imshow(images[2]);    
    plt.axis('on')        
    if deg==0:
        plt.title(titles[2] + ' ' +'steer:'+ str(np.round(steers[2],2) ) );        
    else:
        plt.title('Steer:'+ str((np.round((steer-.1)*180/np.pi,2) )))     

def process_images(image_path,steer,lane_id, command_q):

    image = cv2.imread(image_path)   
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) 

    steer = float(steer)

    while command_q.qsize() > 0:
        command = command_q.get()

		if  command == commands['none']:
			command_q.empty()    

		if  command == commands['shift']:
			image,steer,tr_x = _shift_image(image,steer,lane_id)

		if  command == commands['resize']:
			image = _crop_and_resize_image(image)

		if  command == commands['flip']:
			image,steer = _flip_image(image,steer)

		if  command == commands['brightness']:
			image = _brightness_image(image)

	return image,steer

In [ ]:

# queen_command.put(commands['none'])
queen_command.put(commands['shift'])
# queen_command.put(commands['brightness'])
# queen_command.put(commands['resize'])
# queen_command.put(commands['flip'])
image_c,steer_c = process.process_images(imagefile_path_c,steer,'center', queen_command)

# queen_command.put(commands['none'])
queen_command.put(commands['shift'])
# queen_command.put(commands['brightness'])
# queen_command.put(commands['resize'])
# queen_command.put(commands['flip'])
image_l,steer_l = process.process_images(imagefile_path_l,steer,'left', queen_command)

# queen_command.put(commands['none'])
queen_command.put(commands['shift'])
# queen_command.put(commands['brightness'])
# queen_command.put(commands['resize'])
# queen_command.put(commands['flip'])
image_r,steer_r = process.process_images(imagefile_path_r,steer,'right', queen_command)

image_arr = [image_l,image_c,image_r] 
# steer_arr = [steer_l,steer_c,steer_r]
steer_arr = np.array([steer_l,steer_c,steer_r], dtype='float32')
title_arr = ['Left','Center','Right']
         
plt.rcParams['figure.figsize'] = (12, 6)
process.plot_images(image_arr,steer_arr,title_arr)
plt.show()